In [1]:
import numpy as np
import xarray as xr
import dask
import cftime
import os
import intake
import pandas as pd
from collections import defaultdict
from tqdm.autonotebook import tqdm
import cftime
import gcsfs
from datetime import datetime
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_8500/534830003.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Configure the script:

In [2]:
def get_ddict_of_model_datasets(input_path,ssps):
    #input path must contain directories organized by model
    ddict = defaultdict(dict)
    for model_path in tqdm(fs.ls(input_path)):
        model_datasets = []
        model_ssps = []
        #print(model_path)
        for s,ssp in enumerate(ssps):
            #print(ssp)
            model_fns = ['gs://'+k for k in fs.ls(model_path) if ssp in k]
            if len(model_fns)>0:
                model_ds = xr.open_mfdataset(model_fns,engine='zarr',combine='nested',coords='minimal',compat='override')
                model_datasets.append(model_ds)
                model_ssps.append(ssp)
        
        if len(model_ssps)>0:
            model_ssps_ds = xr.concat(model_datasets,dim='ssp',compat='override',coords='minimal')
            model_ssps_ds['ssp'] = model_ssps
        
            ddict[str(model_ssps_ds.source_id)]=model_ssps_ds
    return ddict

def add_years_months_to_coords(ds):
    years = []
    months = []
    for k,ts in enumerate(ds.time.values):
        try:
            years.append(ts.year)
            months.append(ts.month)
        except:
            years.append(ts.year)
            months.append(ts.month)
    ds = ds.assign_coords({'year':('time',years),'month':('time',months)})
    return ds
    
def derive_dict_of_month_means(ddict,hist,fut):
    month_means = defaultdict(dict)
    for key,ds in ddict.items():

        try:
            ds_hist = ds.sel(time=slice(str(hist[0]),str(hist[-1])))
            ds_fut = ds.sel(time=slice(str(fut[0]),str(fut[-1])))
            month_means[key] = xr.concat((ds_hist.groupby(ds_hist.time.dt.month).mean(),
                                       ds_fut.groupby(ds_fut.time.dt.month).mean()
                                       ),dim='period')
            month_means[key]['period'] = ['hist','fut']

        except: #irregular calendar, issue with selecting years/months
            print('Cannot select years/months due to time index issues, computing these coordinates manually, for: '+key)
            ds = add_years_months_to_coords(ds)
            ds_hist = ds.where((ds.year >= hist[0])&(ds.year <= hist[1]))
            ds_fut = ds.where((ds.year >= fut[0])&(ds.year <= fut[1]))
            month_means[key] = xr.concat((ds_hist.groupby(ds_hist.month).mean(),
                                       ds_fut.groupby(ds_fut.month).mean()
                                       ),dim='period')
            month_means[key]['period'] = ['hist','fut']
            
        #except:
        #    print(key)
        #    print('issue with time units, to sort out')
    return month_means

In [3]:
query_var = 'zos' #variables to process
ssps = ['ssp585']
#ssps = ['ssp126','ssp245','ssp370','ssp585'] #SSPs to process #(TODO: loop over multiple, streamline code!)
input_path = 'gs://leap-persistent/timh37/CMIP6/zos_1x1'
output_path = 'gs://leap-persistent/timh37/CMIP6/'
hist = [1993,2022]
fut = [2071,2100]

Generate dictionary of model datasets:

In [4]:
for ssp in ssps:
    ddict = get_ddict_of_model_datasets(input_path,[ssp]) #open datasets (grouped by model) into dictionary
    month_means = derive_dict_of_month_means(ddict,hist,fut) #compute month means for each dataset
    
    month_means_ds = xr.concat(month_means.values(),dim='source_id') #concatenate datasets along source id
    month_means_ds['source_id'] = list(month_means.keys()) #add source_id's as coordinate
   
    month_means_ds['member_id'] = month_means_ds['member_id'].astype('str') #avoid issue with object dimension type
    month_means_ds.chunk({'source_id':1,'lat':500,'lon':500,'period':2,'member_id':1000}).to_zarr(os.path.join(output_path,query_var+'_monmeans_1x1_hist_'+ssp),mode='w')

  0%|          | 0/40 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/coding/times.py:1001: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/indexing.py:514: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype, copy=copy)
/srv/conda/envs/notebook/lib/python3.12/site-packages/dask/array/core.py:4918: PerformanceWarning: Increasing number of chunks by factor of 10
  result = blockwise(
/srv/conda/envs/notebook/lib/python3.12/site-packages/dask/array/core.py:4918: PerformanceWarning: Increasing number of chunks by factor of 10
  result = blockwise(
/srv/conda/envs/notebook/lib/python3.

Cannot select years/months due to time index issues, computing these coordinates manually, for: ACCESS-CM2
